<center> 
<h1> NLPModels GPU support </h1>
</center>

In [1]:
using LinearAlgebra, Krylov, ForwardDiff
using NLPModels, CUDA

###  Examples

In the field of nonlinear optimization, finding critical points of a continuous function frequently involves linear systems with a Hessian or Jacobian as coefficient. Materializing such operators as matrices is expensive in terms of operations and memory consumption and is unreasonable for high-dimensional problems. However, it is often possible to implement efficient Hessian-vector and Jacobian-vector products, for example with the help of automatic differentiation tools, and used within Krylov solvers. We now illustrate variants with explicit matrices and with matrix-free operators for two well-known optimization methods.

#### Example 1: Newton's Method for convex optimization

At each iteration of Newton's method applied to a $\mathcal{C}^2$ strictly convex function $f : \mathbb{R}^n \rightarrow \mathbb{R}$, a descent direction direction is determined by minimizing the quadratic Taylor model of $f$:
<br><br>
$$
\min_{d \in \mathbb{R}^n}~~f(x_k) + \nabla f(x_k)^T d + \tfrac{1}{2}~d^T \nabla^2 f(x_k) d
$$
<br><br>
which is equivalent to solving the symmetric and positive-definite system
<br><br>
$$
\nabla^2 f(x_k) d  = -\nabla f(x_k).
$$
<br><br>
The system above can be solved with the conjugate gradient method without using the explicit Hessian.
The explicit Hessian can be replaced by a linear operator that only computes Hessian-vector products.

In [2]:
mutable struct MyNLPModel{T, S} <: AbstractNLPModel{T, S}
  meta :: NLPModelMeta{T, S}
  counters :: Counters
  λ :: S
end

In [3]:
function MyNLPModel{T, S}() where {T,S}
    meta = NLPModelMeta{T, S}(5)
  return MyNLPModel{T, S}(meta, Counters(), S([1:5;]))
end

In [4]:
T = Float64
S = CuVector{Float64}
nlp = MyNLPModel{T, S}()

MyNLPModel{Float64, CuArray{Float64, 1}}
  Problem name: Generic
   All variables: ████████████████████ 5      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 5                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   15              linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         

  Counters:
            

In [5]:
function NLPModels.obj(nlp::MyNLPModel, x::AbstractVector)
  increment!(nlp, :neval_obj)
  aux = x .- nlp.λ
  return mapreduce(x -> x^4, +, aux)
end

In [6]:
function NLPModels.grad(nlp::MyNLPModel, x::AbstractVector)
  increment!(nlp, :neval_grad)
  gx = ForwardDiff.gradient(x -> obj(nlp, x), x)
  return gx
end

In [7]:
function NLPModels.hprod!(nlp::MyNLPModel, x::AbstractVector, y::AbstractVector,
                         v::AbstractVector, hv::AbstractVector; obj_weight=1)
  increment!(nlp, :neval_hprod)
  ForwardDiff.derivative!(hv, t -> grad(nlp, x + t * v), 0)
  return hv
end

In [8]:
f(x) = obj(nlp, x)
g(x) = grad(nlp, x)
H(x) = hess_op(nlp, x)

n = 5
xk = -CUDA.ones(T, n)
d, stats = cg(H(xk), -g(xk))

([0.6666666666666666, 1.0, 1.3333333333333333, 1.6666666666666663, 1.9999999999999993], Simple stats
 solved: true
 inconsistent: false
 residuals: []
 Aresiduals: []
 κ₂(A): []
 status: solution good enough given atol and rtol
)

In [9]:
function newton(nlp, x₀; itmax = 200, tol = 1e-8)

    n = length(x₀)
    x = copy(x₀)
    gx = grad(nlp, x)
    
    iter = 0
    S = typeof(x)
    solver = CgSolver(n, n, S)
    Δx = solver.x

    solved = false
    tired = false

    while !(solved || tired)
 
        Hx = hess_op(nlp, x)  # Compute ∇²f(xₖ)
        cg!(solver, Hx, -gx)  # Solve ∇²f(xₖ)Δx = -∇f(xₖ)
        x = x + Δx            # Update xₖ₊₁ = xₖ + Δx
        gx = grad(nlp, x)     # ∇f(xₖ₊₁)
        
        iter += 1
        solved = norm(gx) ≤ tol
        tired = iter ≥ itmax
    end
    return x
end

newton (generic function with 1 method)

In [10]:
x₀ = CUDA.zeros(T, 5)
xstar = newton(nlp, x₀; itmax = 200, tol = 1e-8)

5-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 0.999212671621487
 1.9991489231646358
 2.9991244218266653
 3.999118054977472
 4.999124148570167

#### Example 2: The Gauss-Newton Method for Nonlinear Least Squares

At each iteration of the Gauss-Newton method applied to a nonlinear least-squares objective $f(x) = \tfrac{1}{2}\| F(x)\|^2$ where $F : \mathbb{R}^n \rightarrow \mathbb{R}^m$ is $\mathcal{C}^1$, we solve the subproblem:
$$
\min_{d \in \mathbb{R}^n}~~\tfrac{1}{2}~\|J(x_k) d + F(x_k)\|^2,
$$
where $J(x)$ is the Jacobian of $F$ at $x$.

An appropriate iterative method to solve the above linear least-squares problems is LSMR. We could not pass the explicit Jacobian to LSMR and replace it by a linear operator that only computes Jacobian-vector and transposed Jacobian-vector products.

In [11]:
mutable struct MyNLSModel{T, S} <: AbstractNLSModel{T, S}
  meta::NLPModelMeta{T, S}
  nls_meta::NLSMeta{T, S}
  counters::NLSCounters
  γ::S
end

In [12]:
function MyNLSModel{T, S}() where {T,S}
    meta = NLPModelMeta{T, S}(4)
    nls_meta = NLSMeta{T, S}(4, 4)
    γ = S([i^2 for i=1:4])
  return MyNLSModel{T, S}(meta, nls_meta, NLSCounters(), γ)
end

In [13]:
nls = MyNLSModel{T,S}()

MyNLSModel{Float64, CuArray{Float64, 1}}
  Problem name: Generic
   All variables: ████████████████████ 4      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0        All residuals: ████████████████████ 4     
            free: ████████████████████ 4                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0            nonlinear: ████████████████████ 4     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 nnzj: (  0.00% sparsity)   16    
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 nnzh: (  0.00% sparsity)   10    
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   10              linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅

In [14]:
function residual(nls::MyNLSModel, x::AbstractVector)
    increment!(nls, :neval_residual)
    val = sum(x)
    res = (exp.(x) .- val).^2 + log.(nls.γ)
    return res
end

residual (generic function with 1 method)

In [15]:
function NLPModels.jprod_residual!(nls::MyNLSModel{T,S}, x::S, v::S, Jv::S) where {T,S}
    increment!(nls, :neval_jprod_residual)
    ForwardDiff.derivative!(Jv, t -> residual(nls, x + t * v), 0)
    return Jv
end

In [16]:
function NLPModels.jtprod_residual!(nls::MyNLSModel{T,S}, x::S, v::S, Jtv::S) where {T,S}
    increment!(nls, :neval_jtprod_residual)
    ForwardDiff.gradient!(Jtv, u -> dot(residual(nls, x), u), x)
    return Jtv
end

In [17]:
F(x) = residual(nls, x)
J(x) = jac_op_residual(nls, x)

xk = CUDA.ones(T, 4)
lsmr(J(xk), -F(xk))

([0.21553507435930744, 0.3974164906079825, 0.5038102986915127, 0.579297906856657], Simple stats
 solved: false
 inconsistent: true
 residuals: []
 Aresiduals: []
 κ₂(A): []
 status: maximum number of iterations exceeded
)

In [18]:
function gauss_newton(nls, x₀; itmax = 200, tol = 1e-8)

    n = length(x₀)
    x = copy(x₀)
    Fx = residual(nls, x)
    m = length(Fx)
    
    iter = 0
    S = typeof(x)
    solver = LsmrSolver(m, n, S)
    Δx = solver.x

    solved = false
    tired = false

    while !(solved || tired)
 
        Jx = jac_op_residual(nls, x)  # Compute J(xₖ)
        lsmr!(solver, Jx, -Fx)        # Minimize ‖J(xₖ)Δx + F(xₖ)‖
        x = x + Δx                    # Update xₖ₊₁ = xₖ + Δx
        Fx_old = Fx                   # F(xₖ)
        Fx = residual(nls, x)         # F(xₖ₊₁)
        
        iter += 1
        solved = norm(Fx - Fx_old) / norm(Fx) ≤ tol
        tired = iter ≥ itmax
    end
    return x
end

gauss_newton (generic function with 1 method)

In [19]:
x₀ = CUDA.zeros(T, 4)
xstar = gauss_newton(nls, x₀; itmax = 200, tol = 1e-8)

4-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 44.384560854200444
  0.1950354852682385
 -0.3063049695691704
 -0.6221873150456495